## AFLUX: AFLOW Search-API

**Aim:** Programmatically expose the same functionality as our web search interface at https://aflow.org/search

- AFLUX enables search through the query string of the URL ```?<operator>``` by exposing several logical operators.
- Operators can be inter-property and/or intra-property.

### List of logical operators and AFLUX syntax
|Logical operator |      AFLUX syntax          |        Operator             |
| --------------  |:--------------------------:| ---------------------------:|
| ```<block>```   |    ```(``` and ```)```     |  Intra- and inter-property  |
| ```<AND>```     |          ```,```           |  Intra- and inter-property  |
| ```<OR>```      |          ```:```           |  Intra- and inter-property  |
| ```<NOT>```     |          ```!```           |       Intra-property        |
| ```<loose>```   |          ```*```           |       Intra-property        |
| ```<string>```  |          ```'```           |       Inter-property        |
| ```<mute>```    |          ```$```           |       Intra-property        |

- In AFLUX operators are used to create a query. In our terminology a query is comprised of a **matchbook** and **directives**.

    ![](../images/AFLUX_URL.png)

### Matchbook:
- Materials keywords with arguments: ```<server>?species((Na:K),Cl),nspecies(2),Egap(1*,*5),energy_cell```.

### Directives:
- Used for formatting, moving between layers (catalog) and pagination.
- **format** takes args ```"json"``` and ```"html"```.
- **catalog** takes args ```"icsd"```, ```"lib1"```, ```"lib2"```, ...
- **paging** controls page number, number of entries per page, and sorting. By default entries are sorted in ascending order of first materials keyword. Order can be reversed by using a negative page number.


## AFLUX: URLs

Help directives

In [9]:
# Help text.
!curl 'https://aflow.org/API/aflux/?'

[
    "       Welcome to Aflux - The Aflow search API (version 1.0)                   ",
    "            Aflux home:   https://aflow.org/API/aflux/?                        ",
    "                                                                               ",
    "Aflux is in its first stable release cycle (read, we think it works now).      ",
    "                                                                               ",
    "                                                                               ",
    "The purpose of Aflux is to expose our collection of materials data via         ",
    "arbitrary set based restrictions on any of the properties of interest. By      ",
    "default, matched results are returned as JSON serialized data. An Aflux        ",
    "summons is written entirely in the query portion of the Aflux home URL:        ",
    "        http://aflow.org/API/aflux/?<summons>                                  ",
    "                                    

In [10]:
# List of matchbook keywords.
!curl 'https://aflow.org/API/aflux/?schema,format(json)'

{
    "__schema^2__": {
        "__comment__": [
            "The zeroth element of any object or array in this document is meta.",
            "If last element is null, element parent considered optional.",
            "If last element is '.', element value can be anything.",
            "If last element is '', element value can be nothing.",
            "This document is the AAPI schema, it is self validating and order sensitive.",
            "."
        ],
        "correlated": [
            "For correlated properties, lists the property dependencies.",
            "The counts of the data have to match.",
            "."
        ],
        "delimiter": [
            "An ordered set of single character seperators for distinguishing plural type property values",
            null
        ],
        "description": [
            "intended for popup help boxes, describes the current property: freeform text",
            "."
        ],
        "example": [
            "Actual result that 

        "description": "Returns the entropic temperature.",
        "title": "entropic temperature",
        "format": "%.0f",
        "type": "number",
        "units": "K",
        "inclusion": "mandatory",
        "expression": "derivative",
        "verification": [
            "energy_cutoff",
            "kpoints"
        ],
        "example": "entropic_temperature=1072.1",
        "references": [
            "10.1038/nmat3568",
            "10.1103/PhysRevX.3.041035"
        ],
        "status": "production",
        "syntax": "$aurl/?entropic_temperature"
    },
    "files": {
        "__comment__": [
            ""
        ],
        "description": "Returns the input and output files used in the simulation.",
        "title": "I/O files",
        "format": "%s",
        "type": "strings",
        "delimiter": ",",
        "inclusion": "conditional",
        "expression": "directive",
        "example": "files=Bi_dRh_pv.33.cif,Bi_dRh_pv.33.png,CONTCAR.relax,CONTCAR.relax1,",
  

Operators \<block\>, \<AND\>, \<OR\>, \<NOT\>

In [11]:
# Search compounds containing Cr and 3 total species.
!curl 'https://aflow.org/API/aflux/?species(Cr),catalog(ICSD),nspecies(3),paging(1)'

{"1 of 862":{"compound":"Ag1Cr1O2","auid":"aflow:15d712855422961b","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/RHL/Ag1Cr1O2_ICSD_25624","spacegroup_relax":166,"Pearson_symbol_relax":"hR4","species":["Ag","Cr","O"],"catalog":"ICSD","nspecies":3},"2 of 862":{"compound":"Ag8Cr4O16","auid":"aflow:34ea53770aa1474d","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/ORC/Ag2Cr1O4_ICSD_16298","spacegroup_relax":62,"Pearson_symbol_relax":"oP28","species":["Ag","Cr","O"],"catalog":"ICSD","nspecies":3},"3 of 862":{"compound":"Ag1Cr3O8","auid":"aflow:a9c8e66f03708ad5","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/MCLC/Ag1Cr3O8_ICSD_155507","spacegroup_relax":12,"Pearson_symbol_relax":"mS24","species":["Ag","Cr","O"],"catalog":"ICSD","nspecies":3},"4 of 862":{"compound":"Ag4Cr4O14","auid":"aflow:b260130b38fdc0b5","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/TRI/Ag2Cr2O7_ICSD_2433","spacegroup_relax":2,"Pearson_symbol_relax":"aP22","species":["Ag","Cr","O"],"catalog":"ICSD","nspecies":3},"5 of 862":{"comp

In [12]:
# Exclude oxygen from the results.
!curl 'https://aflow.org/API/aflux/?species(Cr,!O),catalog(ICSD),nspecies(3),paging(1)'

{"1 of 550":{"compound":"Ag1Cr1S2","auid":"aflow:1dcf91da07337d8d","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/RHL/Ag1Cr1S2_ICSD_42396","spacegroup_relax":160,"Pearson_symbol_relax":"hR4","species":["Ag","Cr","S"],"catalog":"ICSD","nspecies":3},"2 of 550":{"compound":"Ag1Cr1S2","auid":"aflow:2f0ba2e2e5753536","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/RHL/Ag1Cr1S2_ICSD_24797","spacegroup_relax":160,"Pearson_symbol_relax":"hR4","species":["Ag","Cr","S"],"catalog":"ICSD","nspecies":3},"3 of 550":{"compound":"Ag1Cr1S2","auid":"aflow:9f664632bc1716e6","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/RHL/Ag1Cr1S2_ICSD_604981","spacegroup_relax":160,"Pearson_symbol_relax":"hR4","species":["Ag","Cr","S"],"catalog":"ICSD","nspecies":3},"4 of 550":{"compound":"Ag1Cr1S2","auid":"aflow:c460cbc83a90f3c9","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/RHL/Ag1Cr1S2_ICSD_42395","spacegroup_relax":160,"Pearson_symbol_relax":"hR4","species":["Ag","Cr","S"],"catalog":"ICSD","nspecies":3},"5 of 550":{"compo

In [13]:
# List compounds from ICSD catalog containing Cr or Mn and 3 species.
!curl 'https://aflow.org/API/aflux/?species(Cr:Mn),catalog(ICSD),nspecies(3),paging(1)'

{"1 of 2378":{"compound":"Ag1Cr1O2","auid":"aflow:15d712855422961b","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/RHL/Ag1Cr1O2_ICSD_25624","spacegroup_relax":166,"Pearson_symbol_relax":"hR4","species":["Ag","Cr","O"],"catalog":"ICSD","nspecies":3},"2 of 2378":{"compound":"Ag8Cr4O16","auid":"aflow:34ea53770aa1474d","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/ORC/Ag2Cr1O4_ICSD_16298","spacegroup_relax":62,"Pearson_symbol_relax":"oP28","species":["Ag","Cr","O"],"catalog":"ICSD","nspecies":3},"3 of 2378":{"compound":"Ag1Cr3O8","auid":"aflow:a9c8e66f03708ad5","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/MCLC/Ag1Cr3O8_ICSD_155507","spacegroup_relax":12,"Pearson_symbol_relax":"mS24","species":["Ag","Cr","O"],"catalog":"ICSD","nspecies":3},"4 of 2378":{"compound":"Ag4Cr4O14","auid":"aflow:b260130b38fdc0b5","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/TRI/Ag2Cr2O7_ICSD_2433","spacegroup_relax":2,"Pearson_symbol_relax":"aP22","species":["Ag","Cr","O"],"catalog":"ICSD","nspecies":3},"5 of 2378":{

74d62477ba333c6","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/FCC/Al1Cr1Cu2_ICSD_57653","spacegroup_relax":225,"Pearson_symbol_relax":"cF16","species":["Al","Cr","Cu"],"catalog":"ICSD","nspecies":3},"51 of 2378":{"compound":"Al8Cr4Dy1","auid":"aflow:1b4889b81cf5a8f3","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/BCT/Al8Cr4Dy1_ICSD_606767","spacegroup_relax":139,"Pearson_symbol_relax":"tI26","species":["Al","Cr","Dy"],"catalog":"ICSD","nspecies":3},"52 of 2378":{"compound":"Al86Cr8Dy12","auid":"aflow:dcb1e8d4cf78bc5b","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/HEX/Al43Cr4Dy6_ICSD_658848","spacegroup_relax":193,"Pearson_symbol_relax":"hP106","species":["Al","Cr","Dy"],"catalog":"ICSD","nspecies":3},"53 of 2378":{"compound":"Al40Cr4Er2","auid":"aflow:5ee7581552c86492","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/FCC/Al20Cr2Er1_ICSD_104794","spacegroup_relax":227,"Pearson_symbol_relax":"cF184","species":["Al","Cr","Er"],"catalog":"ICSD","nspecies":3},"54 of 2378":{"compound":"Al8Cr4Er1","

In [14]:
# Retrieve compounds with 3 species from ICSD catalog containing Cr and Se, or Cr and Te and 3 species.
!curl 'https://aflow.org/API/aflux/?species(Cr,(Se:Te)),catalog(ICSD),nspecies(3),paging(1)'

{"1 of 115":{"compound":"Ag1Cr1Se2","auid":"aflow:0ade16b90ae4e009","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/RHL/Ag1Cr1Se2_ICSD_42397","spacegroup_relax":160,"Pearson_symbol_relax":"hR4","species":["Ag","Cr","Se"],"catalog":"ICSD","nspecies":3},"2 of 115":{"compound":"Ag1Cr1Se2","auid":"aflow:1318e2405f0538fe","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/RHL/Ag1Cr1Se2_ICSD_604993","spacegroup_relax":160,"Pearson_symbol_relax":"hR4","species":["Ag","Cr","Se"],"catalog":"ICSD","nspecies":3},"3 of 115":{"compound":"Ag1Cr1Se2","auid":"aflow:7a186963a851adb4","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/RHL/Ag1Cr1Se2_ICSD_68423","spacegroup_relax":160,"Pearson_symbol_relax":"hR4","species":["Ag","Cr","Se"],"catalog":"ICSD","nspecies":3},"4 of 115":{"compound":"Ag1Cr1Se2","auid":"aflow:8671138d80bd037e","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/RHL/Ag1Cr1Se2_ICSD_25626","spacegroup_relax":160,"Pearson_symbol_relax":"hR4","species":["Ag","Cr","Se"],"catalog":"ICSD","nspecies":3},"5 of 

Operators: \<loose\>

In [15]:
# Retrieve the band gaps for all quaternary compounds
!curl 'https://aflow.org/API/aflux/?Egap,nspecies(4),paging(1)'

{"1 of 509428":{"compound":"Cd1Ir1K1Ru1","auid":"aflow:00eef9c9b73b5e0c","aurl":"aflowlib.duke.edu:AFLOWDATA/LIB4_RAW/CdIrK_svRu_pv:PAW_PBE/ABCD_cF16_216_c_d_b_a.CABD","spacegroup_relax":216,"Pearson_symbol_relax":"cF16","Egap":null,"nspecies":4},"2 of 509428":{"compound":"La1Sr1Ti1Zr1","auid":"aflow:00ee4059fcfd34ce","aurl":"aflowlib.duke.edu:AFLOWDATA/LIB4_RAW/LaSr_svTi_svZr_sv:PAW_PBE/ABCD_cF16_216_c_d_b_a.ABCD","spacegroup_relax":216,"Pearson_symbol_relax":"cF16","Egap":null,"nspecies":4},"3 of 509428":{"compound":"Li1Ru1Sc1Ta1","auid":"aflow:00ee6bf32334886c","aurl":"aflowlib.duke.edu:AFLOWDATA/LIB4_RAW/Li_svRu_pvSc_svTa_pv:PAW_PBE/ABCD_cF16_216_c_d_b_a.BCAD","spacegroup_relax":216,"Pearson_symbol_relax":"cF16","Egap":null,"nspecies":4},"4 of 509428":{"compound":"Hf2Re1Rh4Zr1","auid":"aflow:00eeceed3b527f28","aurl":"aflowlib.duke.edu:AFLOWDATA/LIB4_RAW/Hf_pvRe_pvRh_pvZr_sv:PAW_PBE/A2BC4D_tI16_121_d_a_i_b-001.ABCD","spacegroup_relax":121,"Pearson_symbol_relax":"tI16","Egap":null,"n

In [16]:
# Only show non-null results
!curl 'https://aflow.org/API/aflux/?Egap(*),nspecies(4),paging(1)'

{"1 of 8222":{"compound":"Cs2F6Mn1Na1","auid":"aflow:009f1d82d2db9427","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/FCC/Cs2F6Mn1Na1_ICSD_37005","spacegroup_relax":225,"Pearson_symbol_relax":"cF40","Egap":0,"nspecies":4},"2 of 8222":{"compound":"Al7Cu16H9Zr6","auid":"aflow:00e9a30758f3942a","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/FCC/Al7Cu16H9Zr6_ICSD_76304","spacegroup_relax":225,"Pearson_symbol_relax":"cF152","Egap":0,"nspecies":4},"3 of 8222":{"compound":"Ba2Mn1O6U1","auid":"aflow:01e24d981f1c8d0f","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/FCC/Ba2Mn1O6U1_ICSD_21104","spacegroup_relax":225,"Pearson_symbol_relax":"cF40","Egap":0,"nspecies":4},"4 of 8222":{"compound":"Al8Fe1Mg3Si6","auid":"aflow:011e8cde029b11e5","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/HEX/Al8Fe1Mg3Si6_ICSD_27140","spacegroup_relax":189,"Pearson_symbol_relax":"hP18","Egap":0,"nspecies":4},"5 of 8222":{"compound":"Cu2Hg1S4Sn1","auid":"aflow:01069bda4e78db75","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/BCT/C

In [17]:
# Only show results where Egap is greater than or equal to 1eV
!curl 'https://aflow.org/API/aflux/?Egap(1*),nspecies(4),paging(1)'

{"1 of 6429":{"compound":"Ba6Ni2O18Ru4","auid":"aflow:86c365bc159372d5","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/HEX/Ba3Ni1O9Ru2_ICSD_50831","spacegroup_relax":194,"Pearson_symbol_relax":"hP30","Egap":1,"nspecies":4},"2 of 6429":{"compound":"Br4Cu4O10Te4","auid":"aflow:5b66705058ed3b00","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/TET/Br2Cu2O5Te2_ICSD_152960","spacegroup_relax":81,"Pearson_symbol_relax":"tP22","Egap":1.0003,"nspecies":4},"3 of 6429":{"compound":"Ba6Ni2O18Ru4","auid":"aflow:3ffcd27e8ea1852c","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/HEX/Ba3Ni1O9Ru2_ICSD_69091","spacegroup_relax":194,"Pearson_symbol_relax":"hP30","Egap":1.0036,"nspecies":4},"4 of 6429":{"compound":"Ba4Cu4La4Se12","auid":"aflow:67d2fb8b22a76611","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/ORC/Ba1Cu1La1Se3_ICSD_659177","spacegroup_relax":62,"Pearson_symbol_relax":"oP24","Egap":1.0043,"nspecies":4},"5 of 6429":{"compound":"Cu1O6Sr2W1","auid":"aflow:e4e6b41aca0b016f","aurl":"aflowlib.duke.edu:AFLOWDA

In [18]:
# Only show results where Egap is greater than or equal to 1eV and less than or equal to 3eV
!curl 'https://aflow.org/API/aflux/?Egap(1*,*3),nspecies(4),paging(1)'

{"1 of 3064":{"compound":"Ba6Ni2O18Ru4","auid":"aflow:86c365bc159372d5","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/HEX/Ba3Ni1O9Ru2_ICSD_50831","spacegroup_relax":194,"Pearson_symbol_relax":"hP30","Egap":1,"nspecies":4},"2 of 3064":{"compound":"Br4Cu4O10Te4","auid":"aflow:5b66705058ed3b00","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/TET/Br2Cu2O5Te2_ICSD_152960","spacegroup_relax":81,"Pearson_symbol_relax":"tP22","Egap":1.0003,"nspecies":4},"3 of 3064":{"compound":"Ba6Ni2O18Ru4","auid":"aflow:3ffcd27e8ea1852c","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/HEX/Ba3Ni1O9Ru2_ICSD_69091","spacegroup_relax":194,"Pearson_symbol_relax":"hP30","Egap":1.0036,"nspecies":4},"4 of 3064":{"compound":"Ba4Cu4La4Se12","auid":"aflow:67d2fb8b22a76611","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/ORC/Ba1Cu1La1Se3_ICSD_659177","spacegroup_relax":62,"Pearson_symbol_relax":"oP24","Egap":1.0043,"nspecies":4},"5 of 3064":{"compound":"Cu1O6Sr2W1","auid":"aflow:e4e6b41aca0b016f","aurl":"aflowlib.duke.edu:AFLOWDA

"aflow:e96036cab61a0a9c","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/ORC/Cu2H1O5P1_ICSD_200422","spacegroup_relax":58,"Pearson_symbol_relax":"oP36","Egap":1.0634,"nspecies":4},"61 of 3064":{"compound":"Ba4Na2Ni6O12","auid":"aflow:14d9ddab2cce183b","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/ORCF/Ba2Na1Ni3O6_ICSD_73160","spacegroup_relax":69,"Pearson_symbol_relax":"oF96","Egap":1.0637,"nspecies":4},"62 of 3064":{"compound":"Eu2Li2P2Se8","auid":"aflow:263d04f3e1e0c550","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/ORCC/Eu1Li1P1Se4_ICSD_280199","spacegroup_relax":40,"Pearson_symbol_relax":"oS28","Egap":1.0638,"nspecies":4},"63 of 3064":{"compound":"I6Mo1O18Rb2","auid":"aflow:f8d5d3901b18d810","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/RHL/I6Mo1O18Rb2_ICSD_413880","spacegroup_relax":2,"Pearson_symbol_relax":"aP27","Egap":1.0639,"nspecies":4},"64 of 3064":{"compound":"Cu4Pr4Se12Sr4","auid":"aflow:0a3425d6bfec296e","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/ORC/Cu1Pr1Se3Sr1_ICSD_414181"

Directives: paging

In [19]:
# Show nth (e.g. 5th) results page (default: 64 entries/page):
!curl 'https://aflow.org/API/aflux/?Egap(1*,*3),nspecies(4),paging(5)'

{"257 of 3064":{"compound":"As16K24Nb4Tl4","auid":"aflow:dd9e586c466e3477","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/ORC/As4K6Nb1Tl1_ICSD_281025","spacegroup_relax":62,"Pearson_symbol_relax":"oP48","Egap":1.204,"nspecies":4},"258 of 3064":{"compound":"Nd2O2P2Zn2","auid":"aflow:58565733bbdce76f","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/RHL/Nd1O1P1Zn1_ICSD_85776","spacegroup_relax":166,"Pearson_symbol_relax":"hR8","Egap":1.2045,"nspecies":4},"259 of 3064":{"compound":"Ba4Cu4La4Se12","auid":"aflow:c0bd1c30805e4b44","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/ORC/Ba1Cu1La1Se3_ICSD_78175","spacegroup_relax":62,"Pearson_symbol_relax":"oP24","Egap":1.2045,"nspecies":4},"260 of 3064":{"compound":"Ba4K4P4Se16","auid":"aflow:292257d6089c6c2c","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/ORC/Ba1K1P1Se4_ICSD_414637","spacegroup_relax":62,"Pearson_symbol_relax":"oP28","Egap":1.2057,"nspecies":4},"261 of 3064":{"compound":"Ag2K4S8V2","auid":"aflow:37047c6b0be653e2","aurl":"aflowlib.duke.edu:

In [20]:
# Change page size to m (e.g. 100) entries per page:
!curl 'https://aflow.org/API/aflux/?Egap(1*,*3),nspecies(4),paging(5,100)'

{"401 of 3064":{"compound":"Cu2Nd6Se14Si2","auid":"aflow:30519b002b3b92ac","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/HEX/Cu1Nd3Se7Si1_ICSD_156284","spacegroup_relax":173,"Pearson_symbol_relax":"hP24","Egap":1.295,"nspecies":4},"402 of 3064":{"compound":"Cu8Mo4O28Se4","auid":"aflow:f491e147d368971e","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/MCL/Cu2Mo1O7Se1_ICSD_249957","spacegroup_relax":14,"Pearson_symbol_relax":"mP44","Egap":1.2963,"nspecies":4},"403 of 3064":{"compound":"Bi2Br1Gd1O4","auid":"aflow:4f13099930772b21","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/TET/Bi2Br1Gd1O4_ICSD_92415","spacegroup_relax":123,"Pearson_symbol_relax":"tP8","Egap":1.2967,"nspecies":4},"404 of 3064":{"compound":"C20Co8O20S4","auid":"aflow:6c776639cda38735","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/MCL/C10Co4O10S2_ICSD_4014","spacegroup_relax":14,"Pearson_symbol_relax":"mP52","Egap":1.2985,"nspecies":4},"405 of 3064":{"compound":"Ba2La1O6Ru1","auid":"aflow:987d7aa4670739df","aurl":"aflowlib.duke.

Sorting

In [21]:
# Sort in ascending order:
!curl 'https://aflow.org/API/aflux/?Egap(1*,*3),nspecies(4),$paging(1)'

[{"compound":"Ba6Ni2O18Ru4","auid":"aflow:86c365bc159372d5","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/HEX/Ba3Ni1O9Ru2_ICSD_50831","spacegroup_relax":194,"Pearson_symbol_relax":"hP30","Egap":1,"nspecies":4},{"compound":"Br4Cu4O10Te4","auid":"aflow:5b66705058ed3b00","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/TET/Br2Cu2O5Te2_ICSD_152960","spacegroup_relax":81,"Pearson_symbol_relax":"tP22","Egap":1.0003,"nspecies":4},{"compound":"Ba6Ni2O18Ru4","auid":"aflow:3ffcd27e8ea1852c","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/HEX/Ba3Ni1O9Ru2_ICSD_69091","spacegroup_relax":194,"Pearson_symbol_relax":"hP30","Egap":1.0036,"nspecies":4},{"compound":"Ba4Cu4La4Se12","auid":"aflow:67d2fb8b22a76611","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/ORC/Ba1Cu1La1Se3_ICSD_659177","spacegroup_relax":62,"Pearson_symbol_relax":"oP24","Egap":1.0043,"nspecies":4},{"compound":"Cu1O6Sr2W1","auid":"aflow:e4e6b41aca0b016f","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/BCT/Cu1O6Sr2W1_ICSD_88701","spacegroup_relax":87

In [22]:
# Sort in descending order (also works for n=0):
!curl 'https://aflow.org/API/aflux/?Egap(1*,*3),nspecies(4),$paging(-1)'

[{"compound":"Bi8Li2O28Ta6","auid":"aflow:6f0cba62bdf31d05","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/MCLC/Bi4Li1O14Ta3_ICSD_415141","spacegroup_relax":15,"Pearson_symbol_relax":"mS88","Egap":2.9998,"nspecies":4},{"compound":"Al12Nd4O32Pb8","auid":"aflow:b3df90f005ac3724","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/CUB/Al12Nd4O32Pb8_ICSD_406531","spacegroup_relax":224,"Pearson_symbol_relax":"cP56","Egap":2.9998,"nspecies":4},{"compound":"Cl6La8N2S6","auid":"aflow:19427556657ca06d","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/HEX/Cl3La4N1S3_ICSD_74902","spacegroup_relax":186,"Pearson_symbol_relax":"hP22","Egap":2.9981,"nspecies":4},{"compound":"Ba2Ca1O6Te1","auid":"aflow:24e381c2570de06e","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/FCC/Ba2Ca1O6Te1_ICSD_246112","spacegroup_relax":225,"Pearson_symbol_relax":"cF40","Egap":2.9961,"nspecies":4},{"compound":"Ca4O20Si4Ti4","auid":"aflow:490b2be8c257fb27","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/MCL/Ca1O5Si1Ti1_ICSD_89742","spacegroup_

Aliases

In [23]:
# Search for metal chalcogenides:
!curl 'https://aflow.org/API/aflux/?spacegroup_relax(216),Pearson_symbol_relax(cF8),nspecies(2),species(Chalcogens,Metals),paging(0)'

{"1 of 350":{"compound":"S1Zn1","auid":"aflow:01740b0744259721","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/FCC/S1Zn1_ICSD_77090","spacegroup_relax":216,"Pearson_symbol_relax":"cF8","nspecies":2,"species":["S","Zn"]},"2 of 350":{"compound":"Te3Zn3","auid":"aflow:026b86c6d30dc857","aurl":"aflowlib.duke.edu:AFLOWDATA/LIB2_RAW/TeZn/ICSD_80076.AB","spacegroup_relax":216,"Pearson_symbol_relax":"cF8","nspecies":2,"species":["Te","Zn"]},"3 of 350":{"compound":"Te1Zn1","auid":"aflow:03c3b2cac5f0d3b4","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/FCC/Te1Zn1_ICSD_43713","spacegroup_relax":216,"Pearson_symbol_relax":"cF8","nspecies":2,"species":["Te","Zn"]},"4 of 350":{"compound":"Hg1Se1","auid":"aflow:031274b4026ee141","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/FCC/Hg1Se1_ICSD_639200","spacegroup_relax":216,"Pearson_symbol_relax":"cF8","nspecies":2,"species":["Hg","Se"]},"5 of 350":{"compound":"Hg1S1","auid":"aflow:04a1733998e32c93","aurl":"aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/FCC/Hg1S1_ICSD_6

## AFLUX: Example
The following code block enables the quick interaction with AFLUX using python.

In [24]:
import json
from json import JSONDecodeError
from urllib.request import urlopen

SERVER="https://aflow.org"
API="/API/aflux/?"

# Download a AFLUX response and return it as list of dictionaries
def aflux_request(matchbook, paging=1, no_directives=False):
    request_url = SERVER + API + matchbook
    if not no_directives:
        request_url += f",$paging({paging}),format(json)"
    server_response = urlopen(request_url)
    response_content  = server_response.read().decode("utf-8")
    # Basic error handling
    if server_response.getcode() == 200:
        try:
            return json.loads(response_content)
        except JSONDecodeError:
            pass
    print("AFLUX request failed!")
    print(f"  URL: {request_url}")
    print(f"  Response: {response_content}")
    return []

# Print the build in help of AFLUX
def aflux_help(keyword=None):
    if keyword is None:
        # General help (https://aflow.org/API/aflux/?)
        help_data = aflux_request("", no_directives=True)
        print("\n".join(help_data))
    else:
        # Help regarding a specific keyword (https://aflow.org/API/aflux/?help(keyword))
        help_data = aflux_request(f"help({keyword})")
        for key, entry in help_data.items():
            print(key)
            print(f"  description: {entry['description']}")
            print(f"  units: {entry['units']}")
            print(f"  status: {entry['status']}")
            comment = "\n    ".join(entry["__comment__"]).strip()
            if comment:
                print(f"  comment:\n    {comment}")

From the ICSD catalog, find the VRH bulk moduli ('ael_bulk_modulus_vrh') for materials containing Ti in the AFLOW database. What is the material with the highest bulk modulus? What is the lattice type ('Bravais_lattice_relax') and space group of this material? Is it a metal or an insulator ('Egap_type')? 

In [25]:
aflux_help('ael_bulk_modulus_vrh')

ael_bulk_modulus_vrh
  description: Returns the bulk modulus calculated, using the Voigt-Reuss-Hill average, by AEL.
  units: GPa
  status: production


In [26]:
data_vrh = aflux_request('ael_bulk_modulus_vrh,species(Ti),catalog(icsd),Bravais_lattice_relax,Egap,Egap_type', -1)

In [27]:
print(f"The {data_vrh[0]['Egap_type']} {data_vrh[0]['compound']} ({data_vrh[0]['auid']}) has the highest bulk modulus at {{data_vrh[0]['auid']}} GPa.")
print(f"  lattice: {data_vrh[0]['Bravais_lattice_relax']}")
print(f"  spacegroup: {data_vrh[0]['spacegroup_relax']}")

The metal Ir3Ti1 (aflow:6138a1f196c1c245) has the highest bulk modulus at {data_vrh[0]['auid']} GPa.
  lattice: CUB
  spacegroup: 221
